In [32]:
import pandas as pd
from pathlib import Path
import re
import numpy as np

In [12]:
example_path = Path('/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2')

In [ ]:
class Sequence:
    def __init__(self, path: Path):
        self.path = path
        self.filtered_dir = self.path / "filtered_2D"
        self.samples = []
        # anything else we need to store?
    
    def generate_samples(self):

        # this will return a list(or other datastructure?) of Sample objects
        samples_list = []
        # first we need to load the dataframe for each camera file

        # get number of cameras properly
        num_cameras = 4 # (count number of camN files in the supplied directory)

        for file in self.filtered_dir.glob("cam*.h5"):
            match = re.search(r'(cam\d+)', file.name)
            if not match:
                print(f"Warning: Could not parse camera ID from {file.name}")
                continue

            cam_key = match.group(1)  # e.g. "cam3"
            print(f"Loading filtered detections from {file} ({cam_key})")

            df = pd.read_hdf(file, key="df_with_missing")

            print(df)
            # drop the uppermost level because we don't need it
            new_df = df.copy()
            new_df.columns = new_df.columns.droplevel(0)

            print("DROPPED COLUMNS")

            print(df)

            # go through the frames
            num_frames = len(df)

            bodyparts = df.columns.get_level_values("bodyparts").unique()
            num_bodyparts = len(bodyparts)

            # we know shape of tensor (C, J, 3)
            for i in range(num_frames):
                tensor = np.zeros((num_cameras, num_bodyparts, 3), dtype=np.float32)

                for bodypart in bodyparts:
                    u = df[(bodypart, 'x')].iloc[i]
                    v = df[(bodypart, 'y')].iloc[i]
                    c = df[(bodypart, 'likelihood')].iloc[i]

                    tensor[cam_key, bodypart] = [u, v, c]

                    samples_list.append(tensor)
        

            




            


   
            # I want to read the dataframe and store as a tensor of shape (C, J, 3) where C is the number of cameras, J is the number of joints, and 3 is the (x,y,confidence) for each joint


        # one row of our dataframe, convert it to our "sample" object (whatever that may be)
#         for row in parsedfiles:
#             sample = Sample()
#             self.samples.append(sample)
    
#     def load_from_files(self):
#         # For each camera, load the undistorted 2D detections
#         self.filtered_detections = pd.DataFrame()  # Initialize as empty DataFrame

#         for file in self.filtered_dir.glob("cam*.h5"):
#             match = re.search(r'(cam\d+)', file.name)
#             if not match:
#                 print(f"Warning: Could not parse camera ID from {file.name}")
#                 continue

#             cam_key = match.group(1)  # e.g. "cam3"
#             print(f"Loading filtered detections from {file} ({cam_key})")

#             df = pd.read_hdf(file, key="df_with_missing")
#             df['camera'] = cam_key
#             df['frame'] = df['coords']

#             # we need to have 1 df per camera so num-cam dfs per file, then join them all into one df
#             self.filtered_detections = pd.concat([self.filtered_detections, df], ignore_index=True)





# class Sample:
#     def __init__(self, detections_2d, camera_proj, gt_3d):
#         self.detections_2d = detections_2d
#         self.camera_proj = camera_proj
#         self.gt_3d = gt_3d
    


                 
                 
        #           sequence: Path):
        # self.sequence = sequence
        # self.filtered_dir    = sequence / "filtered_2D"
        # self.gt_3d_path    = sequence / "gt_3D.pickle"

        # # these will be populated by load_from_files()
        # self.filtered_detections    = None
        # self.gt_3d                  = None






In [40]:
test = Sequence(example_path)

In [41]:
test.generate_samples()

Loading filtered detections from /gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam3DLC_resnet152_CheetahOct14shuffle1_500000.h5 (cam3)
scorer    DLC_resnet152_CheetahOct14shuffle1_500000                         \
bodyparts                                     r_eye                          
coords                                            x           y likelihood   
0                                        466.780748  496.310020   0.843646   
1                                        476.662960  496.780764   0.867980   
2                                        467.291212  496.760568   0.816340   
3                                        467.653452  496.879998   0.833451   
4                                        476.905672  496.717315   0.845231   
..                                              ...         ...        ...   
316                                             NaN         NaN   0.031859   
317                                             NaN       

/tmp/ipykernel_345563/1462866564.py:30: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnns = df.columns.droplevel(0)


KeyError: ('r_eye', 'x')

In [31]:
display(test.filtered_detections)

scorer    DLC_resnet152_CheetahOct14shuffle1_500000                         \
bodyparts                                     r_eye                          
coords                                            x           y likelihood   
0                                        466.780748  496.310020   0.843646   
1                                        476.662960  496.780764   0.867980   
2                                        467.291212  496.760568   0.816340   
3                                        467.653452  496.879998   0.833451   
4                                        476.905672  496.717315   0.845231   
...                                             ...         ...        ...   
1279                                            NaN         NaN   0.004362   
1280                                            NaN         NaN   0.000308   
1281                                            NaN         NaN   0.000235   
1282                                            NaN         NaN   0.001993   
1283                                            NaN         NaN   0.000256   

scorer                                                                \
bodyparts       l_eye                         r_shoulder               
coords              x           y likelihood           x           y   
0          477.761635  497.109234   0.865188  468.364375  505.838816   
1          477.798796  497.308653   0.812952  467.206840  513.339016   
2          477.738068  497.299124   0.826227  467.445744  513.202022   
3          477.482832  497.376827   0.834253  467.456618  505.628165   
4          477.585308  497.492764   0.901569  478.050518  513.924471   
...               ...         ...        ...         ...         ...   
1279              NaN         NaN   0.000531         NaN         NaN   
1280              NaN         NaN   0.000151         NaN         NaN   
1281              NaN         NaN   0.000129         NaN         NaN   
1282              NaN         NaN   0.000844         NaN         NaN   
1283              NaN         NaN   0.000096         NaN         NaN   

scorer                             ...                                     \
bodyparts            r_front_knee  ...   tail_base                          
coords    likelihood            x  ...           x           y likelihood   
0           0.648893   477.975610  ...  458.291532  496.340185   0.779278   
1           0.706509   467.915626  ...  458.457967  496.305668   0.708053   
2           0.631165   467.901369  ...  458.221192  496.353792   0.559946   
3           0.480500   467.914589  ...  467.733226  496.262832   0.499875   
4           0.476722   467.821035  ...  467.384385  496.280282   0.394748   
...              ...          ...  ...         ...         ...        ...   
1279        0.001694          NaN  ...         NaN         NaN   0.000390   
1280        0.000370          NaN  ...         NaN         NaN   0.000315   
1281        0.000366          NaN  ...         NaN         NaN   0.000344   
1282        0.000310          NaN  ...         NaN         NaN   0.000905   
1283        0.000324          NaN  ...         NaN         NaN   0.000681   

scorer                                                                \
bodyparts        nose                          neck_base               
coords              x           y likelihood           x           y   
0          467.133634  504.481726   0.874825  477.818254  497.394294   
1          478.029599  505.899045   0.845803         NaN         NaN   
2          467.599322  505.047461   0.829336         NaN         NaN   
3          467.399360  505.536394   0.850702         NaN         NaN   
4          477.986238  506.087321   0.885310         NaN         NaN   
...               ...         ...        ...         ...         ...   
1279              NaN         NaN   0.001492         NaN         NaN   
1280              NaN         NaN   0.000268         NaN         NaN   
1281              NaN      